# Peer Graded Assignment: Segmenting and Clustering Neighborhoods in Toronoto

## Installing Packages Necessary for Scrapping Website Data

In [1]:
! pip install beautifulsoup4
! pip install lxml
! pip install html5lib
! pip install requests

## Importing Necessary Libraries

In [2]:
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Getting Data from Wikipedia Page

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table', class_='wikitable sortable')

# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods


,PostalCode,Borough,Neighborhood


## Populating the dataframe

In [4]:
for tr in table.find_all('tr')[1:]:
    tr_splitted = tr.text.split('\n')
    postal_code= tr_splitted[1]
    borough= tr_splitted[2]
    neighborhood_name = tr_splitted[3]
    if (tr_splitted[2] != 'Not assigned'):
        if (any(neighborhoods.PostalCode == postal_code)):
            loc_index = neighborhoods.loc[neighborhoods['PostalCode']==postal_code].index[0]
            neighborhoods.loc[loc_index, 'Neighborhood'] = neighborhoods.loc[loc_index, 'Neighborhood'] + ', {}'.format(neighborhood_name)
        else:
            if (tr_splitted[2] != 'Not assigned'):
                neighborhood_name = borough
                neighborhoods = neighborhoods.append({'PostalCode': postal_code, 'Borough': borough, 'Neighborhood': neighborhood_name}, ignore_index=True)
            else:
                neighborhoods = neighborhoods.append({'PostalCode': postal_code, 'Borough': borough, 'Neighborhood': neighborhood_name}, ignore_index=True)
                
   

In [5]:
# Number of rows of the dataframe
neighborhoods.shape[0]

103

In [6]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,North York
1,M4A,North York,North York
2,M5A,Downtown Toronto,"Downtown Toronto, Regent Park"
3,M6A,North York,"North York, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [7]:
neighborhoods['Latitude']=''
neighborhoods['Longitude'] =''
df_latlong = pd.read_csv('Geospatial_Coordinates.csv')
df_latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
for idx, row in neighborhoods.iterrows():
    #print(idx, row['PostalCode'], type(row['PostalCode']))
    postal_code = row['PostalCode']
    loc_= df_latlong.loc[df_latlong['Postal Code']==postal_code].index[0]
    lat = df_latlong.loc[loc_,'Latitude']
    long = df_latlong.loc[loc_,'Longitude']
    neighborhoods.loc[idx,'Latitude'] = lat
    neighborhoods.loc[idx,'Longitude'] = long
    

In [9]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,North York,43.7533,-79.3297
1,M4A,North York,North York,43.7259,-79.3156
2,M5A,Downtown Toronto,"Downtown Toronto, Regent Park",43.6543,-79.3606
3,M6A,North York,"North York, Lawrence Manor",43.7185,-79.4648
4,M7A,Queen's Park,Queen's Park,43.6623,-79.3895


In [10]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6524203, -79.3834045.


## Create a map of of Toronto with neighborhoods superimposed on it

In [11]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Creating New Dataframe with Borough containing "Toronto" 

In [12]:
toronto_data = neighborhoods[~(neighborhoods['Borough'].str.find('Toronto') == -1)].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Downtown Toronto, Regent Park",43.6543,-79.3606
1,M5B,Downtown Toronto,"Downtown Toronto, Garden District",43.6572,-79.3789
2,M5C,Downtown Toronto,Downtown Toronto,43.6515,-79.3754
3,M4E,East Toronto,East Toronto,43.6764,-79.293
4,M5E,Downtown Toronto,Downtown Toronto,43.6448,-79.3733


In [13]:
# create map of Toronto using latitude and longitude values
map_onlyToronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_onlyToronto)  
    
map_onlyToronto

## Foursquare Credentials

In [14]:
CLIENT_ID = '024THNSYKYCB1ZIGBB0XZSJHU0LSVOI00LJ3TKMTZ1W4K5QT' # your Foursquare ID
CLIENT_SECRET = 'IISKVVH2SV4ZJNXWG5V0NBXJICEK4MIJFW0QTWWHISXZDL5B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 024THNSYKYCB1ZIGBB0XZSJHU0LSVOI00LJ3TKMTZ1W4K5QT
CLIENT_SECRET:IISKVVH2SV4ZJNXWG5V0NBXJICEK4MIJFW0QTWWHISXZDL5B


In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Downtown Toronto, Regent Park
Downtown Toronto, Garden District
Downtown Toronto
East Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto, King, Richmond
West Toronto, Dufferin
Downtown Toronto, Toronto Islands, Union Station
West Toronto, Trinity
East Toronto, Riverdale
Downtown Toronto, Toronto Dominion Centre
West Toronto, Exhibition Place, Parkdale Village
East Toronto, India Bazaar
Downtown Toronto, Victoria Hotel
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto, Forest Hill West
West Toronto, The Junction South
Central Toronto
Central Toronto, North Midtown, Yorkville
West Toronto, Roncesvalles
Central Toronto
Downtown Toronto, University of Toronto
West Toronto, Swansea
Central Toronto, Summerhill East
Downtown Toronto, Grange Park, Kensington Market
Central Toronto, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Downtown Toronto, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South N

In [18]:
print(toronto_venues.shape)
toronto_venues.head()

(1712, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Downtown Toronto, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Downtown Toronto, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Downtown Toronto, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Downtown Toronto, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Downtown Toronto, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [19]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,73,73,73,73,73,73
"Central Toronto, Forest Hill SE, Rathnelly, South Hill, Summerhill West",15,15,15,15,15,15
"Central Toronto, Forest Hill West",4,4,4,4,4,4
"Central Toronto, North Midtown, Yorkville",25,25,25,25,25,25
"Central Toronto, Summerhill East",4,4,4,4,4,4
Downtown Toronto,442,442,442,442,442,442
"Downtown Toronto, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Downtown Toronto, Garden District",100,100,100,100,100,100
"Downtown Toronto, Grange Park, Kensington Market",100,100,100,100,100,100


In [20]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 242 uniques categories.


In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Downtown Toronto, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1

In [22]:
toronto_onehot.shape

(1712, 242)

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Central Toronto,0.013699,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.013699,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013699,0.013699,0.000000,0.00,0.027397,0.000000,0.013699,0.000000,0.000000,0.027397,0.000000,0.000000,0.000000,0.000000,0.027397,0.000000,0.000000,0.000000,0.041096,0.000000

In [24]:
toronto_grouped.shape

(26, 242)

In [25]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0  Sandwich Place  0.07
1            Park  0.05
2    Dessert Shop  0.05
3     Coffee Shop  0.05
4  Clothing Store  0.04


----Central Toronto, Forest Hill SE, Rathnelly, South Hill, Summerhill West----
               venue  freq
0        Coffee Shop  0.13
1                Pub  0.13
2  Convenience Store  0.07
3        Pizza Place  0.07
4         Sports Bar  0.07


----Central Toronto, Forest Hill West----
              venue  freq
0     Jewelry Store  0.25
1             Trail  0.25
2              Park  0.25
3  Sushi Restaurant  0.25
4     Movie Theater  0.00


----Central Toronto, North Midtown, Yorkville----
            venue  freq
0  Sandwich Place  0.12
1            Café  0.12
2     Coffee Shop  0.12
3     Pizza Place  0.08
4  Cosmetics Shop  0.04


----Central Toronto, Summerhill East----
          venue  freq
0    Playground  0.25
1           Gym  0.25
2    Restaurant  0.25
3  Tennis Court  0.25
4          Park  0.00


----Downtown To

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10
import numpy as np
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Sandwich Place,Dessert Shop,Park,Coffee Shop,Clothing Store,Pizza Place,Gym / Fitness Center,Sushi Restaurant,Italian Restaurant,Sporting Goods Shop
1,"Central Toronto, Forest Hill SE, Rathnelly, So...",Coffee Shop,Pub,Fried Chicken Joint,Sushi Restaurant,Bagel Shop,Sports Bar,American Restaurant,Medical Center,Supermarket,Convenience Store
2,"Central Toronto, Forest Hill West",Trail,Park,Jewelry Store,Sushi Restaurant,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store
3,"Central Toronto, North Midtown, Yorkville",Café,Coffee Shop,Sandwich Place,Pizza Place,Indian Restaurant,BBQ Joint,Jewish Restaurant,Pub,Cheese Shop,Burger Joint
4,"Central Toronto, Summerhill East",Restaurant,Gym,Playground,Tennis Court,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant


## Clustering Neighborhoods of Toronto

In [28]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1) # first coloumn of the data frame i.e. neighborhood is removed

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([1, 4, 0, 1, 2, 4, 3, 1, 1, 4], dtype=int32)

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Downtown Toronto, Regent Park",43.6543,-79.3606,4,Coffee Shop,Park,Café,Pub,Bakery,Breakfast Spot,Mexican Restaurant,Theater,Ice Cream Shop,Chocolate Shop
1,M5B,Downtown Toronto,"Downtown Toronto, Garden District",43.6572,-79.3789,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Restaurant,Diner,Ramen Restaurant,Pizza Place,Bubble Tea Shop
2,M5C,Downtown Toronto,Downtown Toronto,43.6515,-79.3754,4,Coffee Shop,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Park,Hotel,Seafood Restaurant,Breakfast Spot,Bakery
3,M4E,East Toronto,East Toronto,43.6764,-79.293,1,Coffee Shop,Café,Yoga Studio,Bakery,Light Rail Station,American Restaurant,Italian Restaurant,Brewery,Gym / Fitness Center,Gastropub
4,M5E,Downtown Toronto,Downtown Toronto,43.6448,-79.3733,4,Coffee Shop,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Park,Hotel,Seafood Restaurant,Breakfast Spot,Bakery


In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine the Clusters

## Cluster-1

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Central Toronto,0,Trail,Park,Jewelry Store,Sushi Restaurant,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store


## Cluster-2

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Restaurant,Diner,Ramen Restaurant,Pizza Place,Bubble Tea Shop
3,East Toronto,1,Coffee Shop,Café,Yoga Studio,Bakery,Light Rail Station,American Restaurant,Italian Restaurant,Brewery,Gym / Fitness Center,Gastropub
8,West Toronto,1,Pharmacy,Supermarket,Discount Store,Bakery,Fast Food Restaurant,Liquor Store,Middle Eastern Restaurant,Music Venue,Park,Pizza Place
10,West Toronto,1,Bar,Coffee Shop,Asian Restaurant,Bakery,Pizza Place,Men's Store,Vietnamese Restaurant,New American Restaurant,Cocktail Bar,Restaurant
14,East Toronto,1,Park,Sandwich Place,Pet Store,Brewery,Burger Joint,Burrito Place,Pub,Pizza Place,Movie Theater,Sushi Restaurant
16,East Toronto,1,Coffee Shop,Café,Yoga Studio,Bakery,Light Rail Station,American Restaurant,Italian Restaurant,Brewery,Gym / Fitness Center,Gastropub
17,Central Toronto,1,Sandwich Place,Dessert Shop,Park,Coffee Shop,Clothing Store,Pizza Place,Gym / Fitness Center,Sushi Restaurant,Italian Restaurant,Sporting Goods Shop
18,Central Toronto,1,Sandwich Place,Dessert Shop,Park,Coffee Shop,Clothing Store,Pizza Place,Gym / Fitness Center,Sushi Restaurant,Italian Restaurant,Sporting Goods Shop
19,Central Toronto,1,Sandwich Place,Dessert Shop,Park,Coffee Shop,Clothing Store,Pizza Place,Gym / Fitness Center,Sushi Restaurant,Italian Restaurant,Sporting Goods Shop
21,West Toronto,1,Bar,Mexican Restaurant,Café,Fried Chicken Joint,Antique Shop,Cajun / Creole Restaurant,Diner,Italian Restaurant,Arts & Crafts Store,Furniture / Home Store


## Cluster-3

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Central Toronto,2,Restaurant,Gym,Playground,Tennis Court,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant


## Cluster-4

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Downtown Toronto,3,Airport Lounge,Airport Terminal,Airport Service,Plane,Sculpture Garden,Boutique,Boat or Ferry,Harbor / Marina,Airport Gate,Airport


## Cluster-5

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,4,Coffee Shop,Park,Café,Pub,Bakery,Breakfast Spot,Mexican Restaurant,Theater,Ice Cream Shop,Chocolate Shop
2,Downtown Toronto,4,Coffee Shop,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Park,Hotel,Seafood Restaurant,Breakfast Spot,Bakery
4,Downtown Toronto,4,Coffee Shop,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Park,Hotel,Seafood Restaurant,Breakfast Spot,Bakery
5,Downtown Toronto,4,Coffee Shop,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Park,Hotel,Seafood Restaurant,Breakfast Spot,Bakery
6,Downtown Toronto,4,Coffee Shop,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Park,Hotel,Seafood Restaurant,Breakfast Spot,Bakery
7,Downtown Toronto,4,Coffee Shop,Steakhouse,Café,Thai Restaurant,Bar,Sushi Restaurant,American Restaurant,Hotel,Restaurant,Gym
9,Downtown Toronto,4,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Pizza Place,Brewery,Bakery,Scenic Lookout,Fried Chicken Joint
11,East Toronto,4,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Yoga Studio,Spa,Bookstore,Brewery,Bubble Tea Shop,Restaurant
12,Downtown Toronto,4,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Gastropub,Italian Restaurant,Seafood Restaurant,Deli / Bodega,Pizza Place
13,West Toronto,4,Coffee Shop,Breakfast Spot,Café,Yoga Studio,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Pet Store,Bar


## Conclusions

The K means clustering algorithm is run varying the number of clusters. For 5 clusters, data is mainly clustered into two clusters as shown above, and other clusters contained one or two data points. In order to see the effect of varying the number of clusters on clustering the data, the number is varied between 2-5. However, it was found that, again, data is mainly clustered into two and increasing the number of clusters does not really affect the distribution of the data i.e. data is mainly grouped into two clusters.

Characteristic of the clusters can be obtained anlyzing the data points in the clusters. However, it is a bit difficult to derive any such characteristic for a cluster containing a large number of data points. In above clusters, cluster-5 is characterised by Coffee Shop as the the 1st common venue in the neighborhoods.